# Creating model

In [11]:
import zipfile
if not os.path.exists('/content/test/new_files/cats/'):
  os.makedirs('/content/imgs/test/new_files/cats/')
if not os.path.exists('/content/imgs/test/new_files/dogs/'):
  os.makedirs('/content/imgs/test/new_files/dogs/')
zip_file = "/content/imgs.zip"
startswith_cats = "cat" # начало имени файла для кошек
startswith_dogs = "dog" # начало имени файла для собак
i = 0 # счетчик для кошек
j = 0 # счетчик для собак
limit = 50
with zipfile.ZipFile(zip_file, 'r') as z:
  for file in z.namelist():
    file_new = file
    if file_new.startswith(startswith_cats) and i < limit:
      i += 1
      z.extract(file, 'imgs/test/')
      os.replace('imgs/test/'+file_new,'imgs/test/new_files/cats/'+file_new)
    elif file_new.startswith(startswith_dogs) and j < limit:
      j += 1
      z.extract(file,'imgs/test/')
      os.replace('imgs/test/'+file_new,'imgs/test/new_files/dogs/'+file_new)
    elif i >= limit and j >= limit :
      break

FileExistsError: ignored

In [12]:
import os
from tensorflow import keras
import json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from keras.utils import plot_model
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw 
import os, random, shutil
from PIL import ImageFile
import zipfile 

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model = keras.models.model_from_json(open('/content/model_10k.json').read())
model.load_weights('/content/Model_10k_images.h5')
model.compile(metrics=['acc',f1_m,precision_m, recall_m])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

# New Section

In [13]:
img_width, img_height = 150, 150
new_files_dir = '/content/imgs/'
new_files_datagen = ImageDataGenerator(rescale=1. / 255)
new_files_generator = new_files_datagen.flow_from_directory(
    new_files_dir,
    target_size=(img_width, img_height),
    batch_size=100,
    class_mode='binary',
    shuffle=False) #Сохраняет последовательноcть файлов из папки

Found 0 images belonging to 1 classes.


In [14]:
imgs,labels = new_files_generator.next()
print(imgs)
for img in imgs:
  print(img_to_array(img))

[]


In [15]:
imgs,labels = new_files_generator.next()
print(img_to_array(img) for img in imgs)

array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions = model.predict(imgs)
print("Predictions:")
print(labels)
print([x for x in predictions[:5]]) # выводим первые 3 предсказания


<generator object <genexpr> at 0x7f31a7efdc50>


ValueError: ignored

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

rounded_pred=np.asarray([np.round(i) for i in predictions]) #  округляем
print("Accuracy: ", end='')
print(accuracy_score(labels, rounded_pred))
print("F1 macro avg: ", end='')
print(round(f1_score(labels, rounded_pred, average='macro'), 3))
print("Recall macro avg: ", end='')
print(round(recall_score(labels, rounded_pred, average='macro'), 3))

NameError: ignored

In [17]:
i = 74
if np.round(predictions[i], 2) <= 0.5:
      print('Это кот', 1 - np.round(predictions[i], 2))
else:
      print('Это собака',np.round(predictions[i]))
plt.imshow(np.transpose(array_imgs[i],(1,0,2)))

NameError: ignored